# The PolInSAR Course - April 29, 2024
# SAR Interferometry (InSAR) 
# Part 1: The interferometric phase

* Etna acquisitions
* Path: 'data/insar/'
* SLC: image1, image2

Objective:
- Implement a full (simplified) processing chain from the SLCs to the interferogram

Tips:
- Use fft2 for coregistration
- Other useful functions: np.unravel_index(index, dims), np.roll(array, shift, axes)

In [ ]:
# --- Download exercise data & import reader function
from pysarpro import io, data
from pysarpro.io import rrat

data.download_all(directory="/projects", pattern=r'^data/insar')

# --- Import useful libraries, functions, and modules
import sys
sys.path.append('/projects/src/')
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import uniform_filter
from numpy.fft import fft2, ifft2

%matplotlib widget

In [ ]:
def calculate_covariance(im1, im2, looksr, looksa) : 
    
    corr = uniform_filter(np.real(im1*np.conj(im2)), [looksa,looksr]) + 1j* \
                uniform_filter(np.imag(im1*np.conj(im2)), [looksa,looksr])
    
    return corr

**Step 1 : Load data**

In [ ]:
# --- input path & images
path = 'data/insar/'
im1name = 'image1.rat'
im2name = 'image2.rat'

# --- multilook window size
looksa = 5
looksr = 5

# --- load data ...
im1 = rrat(path + im1name)
im2 = rrat(path + im2name)

# --- calculate amplitudes
amp1 = np.sqrt( np.abs(calculate_covariance(im1, im1, looksr, looksa)) )
amp2 = np.sqrt( np.abs(calculate_covariance(im2, im2, looksr, looksa)) )

# --- display im1
plt.figure( figsize = (10, 8) )
plt.subplot(2, 1, 1)
plt.imshow(np.transpose(amp1), vmin = 0, vmax = 2.5*np.mean(amp1), aspect = 'auto', cmap = 'gray')
plt.title('Image 1, amplitude')
plt.subplot(2, 1, 2)
plt.imshow(np.transpose(np.angle(im1)), vmin = -np.pi, vmax = np.pi, aspect = 'auto', cmap = 'jet', interpolation = 'nearest')
plt.title('Image 1, phase')

# --- display im2
plt.figure( figsize = (10, 8) )
plt.subplot(2, 1, 1)
plt.imshow(np.transpose(amp2), vmin = 0, vmax = 2.5*np.mean(amp2), aspect = 'auto', cmap = 'gray')
plt.title('Image 2, amplitude')
plt.subplot(2, 1, 2)
plt.imshow(np.transpose(np.angle(im2)), vmin = -np.pi, vmax = np.pi, aspect = 'auto', cmap = 'jet', interpolation = 'nearest')
plt.title('Image 2, phase')


**Step 2 : Calculate interferogram**

In [ ]:
interferogram = calculate_covariance(im1, im2, looksr, looksa)

# display interferogram
plt.figure( figsize = (10, 8) )
plt.subplot(2, 1, 1)
plt.imshow(np.transpose(np.abs(interferogram)), vmin = 0, vmax = 2.5*np.mean(np.abs(interferogram)), aspect = 'auto', cmap = 'gray')
plt.title('Interferogram, absolute value')
plt.subplot(2, 1, 2)
plt.imshow(np.transpose(np.angle(interferogram)), vmin = -np.pi, vmax = np.pi, aspect = 'auto', cmap = 'jet', interpolation = 'nearest')
plt.title('Interferogram, phase')

**Step 3 : Coregistration**

In [ ]:
# --- calculate correlation
ff1 = np.conj( fft2(np.abs(im1)) )
ff2 = fft2(np.abs(im2))
correlation = np.abs( ifft2( ff1 * ff2 ) )

# --- display correlation
plt.figure( figsize = (10, 4) )
plt.imshow(np.transpose(correlation), aspect = 'auto', cmap = 'jet')
plt.title('Correlation')

# --- estimate the shift as the position of the maximum
shift = np.unravel_index( np.argmax(correlation), correlation.shape)
print(shift)

# --- shift compensation
im2_co = np.roll(im2, (-shift[0], -shift[1]), axis = [0, 1])

# --- calculate interferogram
interferogram_co = calculate_covariance(im1, im2_co, looksr, looksa)

# --- display
plt.figure( figsize = (10, 8) )
plt.subplot(2, 1, 1)
plt.imshow(np.transpose(np.abs(interferogram_co)), vmin = 0, vmax = 2.5*np.mean(np.abs(interferogram_co)), aspect = 'auto', cmap = 'gray')
plt.title('Interferogram, absolute value')
plt.subplot(2, 1, 2)
plt.imshow(np.transpose(np.angle(interferogram_co)), vmin = -np.pi, vmax = np.pi, aspect = 'auto', cmap = 'jet', interpolation = 'nearest')
plt.title('Interferogram, phase')

**Step 4 : Calculation & compensation of flat-earth phase**

**Step 5 : Calculate Coherence**